<a href="https://colab.research.google.com/github/pratham-rajesh/recommender-system-hackathon-256/blob/main/Market_Basket_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electronic Item Checkout Web Portal
## Market Basket Recommendation System using Apriori Algorithm

This notebook implements an interactive web portal for market basket analysis with real-time recommendations.

**Features:**
- Add items to cart one by one
- Dynamic recommendations based on Apriori association rules
- Strict matching: only rules where ALL antecedent items are in cart
- Validation panel showing rule details and metrics

## Step 1: Install Dependencies

In [1]:
# Install required packages
!pip install -q pandas streamlit mlxtend pyngrok

print("✅ All dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.3 MB/s eta 0:00:00
✅ All dependencies installed!


## Step 2: Upload CSV Dataset

Upload your `CMPE256_Hackathon_market_basket_analysis_Release.csv` file.

In [ ]:
from google.colab import files
import os

# Upload CSV file
uploaded = files.upload()

# Find the CSV file
csv_file = None
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        csv_file = filename
        break

if csv_file:
    print(f"✅ File uploaded: {csv_file}")
    print(f"📁 File size: {os.path.getsize(csv_file) / 1024:.2f} KB")
else:
    print("⚠️ No CSV file found. Please upload a CSV file.")
    print("Expected filename: CMPE256_Hackathon_market_basket_analysis_Release.csv")

Saving CMPE256_Hackathon_market_basket_analysis_Release.csv to CMPE256_Hackathon_market_basket_analysis_Release.csv
✅ File uploaded: CMPE256_Hackathon_market_basket_analysis_Release.csv
📁 File size: 172.94 KB


## Step 3: Create Streamlit App

In [ ]:
%%writefile app.py
"""
Electronic Item Checkout Web Portal
A Market Basket Recommendation System using Apriori Algorithm
"""

import pandas as pd
import streamlit as st
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

# Page configuration
st.set_page_config(
    page_title="Electronic Item Checkout Web Portal",
    page_icon="🛒",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Custom CSS styling to match the SJSU portal theme
st.markdown("""
<style>
    .main-header {
        background-color: #003767;
        color: white;
        padding: 20px;
        border-radius: 5px;
        margin-bottom: 20px;
    }
    .section-header {
        font-size: 18px;
        font-weight: bold;
        color: #003767;
        margin-top: 20px;
        margin-bottom: 10px;
    }
    .cart-item {
        padding: 12px;
        margin: 8px 0;
        background-color: #ffffff;
        border: 1px solid #d0d0d0;
        border-left: 4px solid #003767;
        border-radius: 5px;
        color: #333333;
        font-size: 14px;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }
    .recommended-item {
        padding: 10px;
        margin: 5px 0;
        background-color: #ffffff;
        border: 1px solid #d0d0d0;
        border-left: 4px solid #28a745;
        border-radius: 5px;
        color: #333333;
        font-size: 14px;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }
    .stButton>button {
        background-color: #28a745;
        color: white;
        font-weight: bold;
        border-radius: 5px;
        border: none;
        padding: 10px 20px;
    }
    .stButton>button:hover {
        background-color: #218838;
    }
</style>
""", unsafe_allow_html=True)

@st.cache_data
def load_and_preprocess_data(file_path):
    """
    Load the CSV file and preprocess data for Apriori algorithm.

    Returns:
        transactions: List of lists containing transaction items
        all_items: List of all unique items in the dataset
    """
    # Read CSV
    df = pd.read_csv(file_path)

    # Combine item_1 through item_5 into lists, ignoring NaNs
    transactions = []
    all_items_set = set()

    for idx, row in df.iterrows():
        transaction = []
        for col in ['item_1', 'item_2', 'item_3', 'item_4', 'item_5']:
            item = row[col]
            if pd.notna(item) and str(item).strip():
                transaction.append(str(item).strip())
                all_items_set.add(str(item).strip())

        if transaction:  # Only add non-empty transactions
            transactions.append(transaction)

    all_items = sorted(list(all_items_set))

    return transactions, all_items

@st.cache_data
def generate_association_rules(transactions, min_support=0.02, min_confidence=0.3):
    """
    Generate association rules using Apriori algorithm.

    Parameters:
        transactions: List of lists containing transaction items
        min_support: Minimum support threshold for frequent itemsets
        min_confidence: Minimum confidence threshold for association rules

    Returns:
        rules_df: DataFrame containing association rules
    """
    # Encode transactions
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    # Generate frequent itemsets using Apriori
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)

    # Generate association rules
    if len(frequent_itemsets) > 0:
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
        return rules
    else:
        return pd.DataFrame()

def recommend_items(cart, rules_df, transactions=None, top_n=3, return_details=False):
    """
    Given a list of items in the cart, return top recommended items
    based on association rules (confidence or lift).

    Parameters:
        cart: List of items currently in the cart
        rules_df: DataFrame containing association rules
        transactions: Optional list of transactions for fallback
        top_n: Number of recommendations to return
        return_details: If True, return detailed info about rules used

    Returns:
        List of recommended items, or if return_details=True: (items, details_dict)
    """
    if len(cart) == 0:
        return [] if not return_details else ([], {})
    if len(rules_df) == 0:
        # No rules, try fallback if available
        if transactions is not None:
            fallback_result = recommend_from_transactions(cart, transactions, top_n, return_counts=return_details)
            if return_details and isinstance(fallback_result, tuple):
                return fallback_result[0], {'fallback_used': True, 'rules_used': {}, 'transaction_counts': fallback_result[1]}
            elif return_details:
                return fallback_result, {'fallback_used': True, 'rules_used': {}, 'transaction_counts': {}}
            else:
                return fallback_result
        return [] if not return_details else ([], {})

    recommendations = {}
    recommendation_details = {}  # Track which rules support each recommendation

    # Create normalized sets and mappings for cart items
    # Use exact item names for strict matching
    cart_set = set(item.strip() for item in cart)
    cart_normalized = {item.lower().strip(): item for item in cart}
    cart_set_lower = set(cart_normalized.keys())

    # Filter rules: only keep rules where ALL antecedent items are in cart (strict subset check)
    valid_rules = []
    for idx, rule in rules_df.iterrows():
        antecedents = rule['antecedents']

        if isinstance(antecedents, frozenset):
            # Convert antecedent items to strings and check if ALL are in cart
            antecedent_items = {str(item).strip() for item in antecedents}

            # STRICT RULE MATCHING: Only include rules where ALL antecedent items
            # are fully contained within the current cart. No partial or fuzzy matching.
            if antecedent_items.issubset(cart_set):
                valid_rules.append((idx, rule, len(antecedent_items)))

    # Early return if no valid rules found (no historical transaction covers all cart items)
    if not valid_rules:
        return [] if not return_details else ([], {
            'rules_used': {},
            'fallback_used': False,
            'fallback_type': None,
            'transaction_counts': {},
            'cart_size': len(cart)
        })

    # DYNAMIC FILTERING: As cart grows, prioritize rules using MORE items from cart
    # For carts with 3+ items: Only use rules with 2+ item antecedents
    # For carts with 1-2 items: Use all matching rules
    cart_size = len(cart)
    if cart_size >= 3:
        # Filter to only rules with 2+ items in antecedent
        valid_rules = [(idx, rule, size) for idx, rule, size in valid_rules if size >= 2]

        # If no multi-item rules, return empty (realistic - no historical pattern matches)
        if not valid_rules:
            return [] if not return_details else ([], {
                'rules_used': {},
                'fallback_used': False,
                'fallback_type': None,
                'transaction_counts': {},
                'cart_size': cart_size
            })

    # Prioritize rules with larger antecedents (more items from cart used together)
    # Sort by antecedent size (descending) so multi-item rules come first
    valid_rules.sort(key=lambda x: x[2], reverse=True)

    # Process only valid rules (sorted by antecedent size, largest first)
    for idx, rule, ant_size in valid_rules:
        antecedents = rule['antecedents']
        consequents = rule['consequents']
        confidence = rule['confidence']
        lift = rule['lift']

        # Extract consequent items
        if isinstance(consequents, frozenset):
                consequent_items = [str(item).strip() for item in consequents]

                for consequent_str in consequent_items:
                    consequent_normalized = consequent_str.lower().strip()

                    # Skip if already in cart
                    in_cart = False

                    # Exact match check
                    if consequent_normalized in cart_set_lower:
                        in_cart = True
                    else:
                        # Partial match check - see if any cart item matches this consequent
                        for cart_norm in cart_set_lower:
                            if (consequent_normalized == cart_norm or
                                consequent_normalized in cart_norm or
                                cart_norm in consequent_normalized):
                                in_cart = True
                                break

                    if not in_cart:
                        # Use weighted score (confidence * lift)
                        score = confidence * lift
                        if consequent_str not in recommendations:
                            recommendations[consequent_str] = score
                            recommendation_details[consequent_str] = []
                        else:
                            # Keep the highest score for duplicate recommendations
                            if score > recommendations[consequent_str]:
                                recommendations[consequent_str] = score

                        # Track the rule that supports this recommendation
                        if return_details:
                            antecedent_list = [str(a) for a in antecedents]
                            # Get support if available (antecedent support)
                            support = rule.get('antecedent support', rule.get('support', 'N/A'))
                            recommendation_details[consequent_str].append({
                                'antecedents': antecedent_list,
                                'confidence': confidence,
                                'lift': lift,
                                'support': support,
                                'conviction': rule.get('conviction', 'N/A')
                            })

    # Sort by score and return top N unique recommendations
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    result = [item for item, score in sorted_recommendations[:top_n]]

    fallback_used = False
    transaction_counts = {}
    fallback_type = None

    # NO FALLBACK STRATEGIES - Only use strict rule matching
    # This ensures recommendations are realistic and narrow down as cart grows
    # If no valid rules remain after filtering, return empty list (handled gracefully in UI)

    if return_details:
        return result, {
            'rules_used': recommendation_details,
            'fallback_used': False,  # No fallbacks used - strict matching only
            'fallback_type': None,
            'transaction_counts': {},
            'cart_size': len(cart)
        }

    return result

def display_recommendation_validation(recommended_items, details, cart):
    """
    Display validation information for recommendations.
    Shows which rules or transaction data support each recommendation.
    """
    if not details:
        st.write("No validation details available.")
        return

    fallback_used = details.get('fallback_used', False)
    fallback_type = details.get('fallback_type', None)
    rules_used = details.get('rules_used', {})
    transaction_counts = details.get('transaction_counts', {})
    cart_size = details.get('cart_size', len(cart))

    if fallback_used:
        if fallback_type == "items_appearing_with_all_cart_items":
            st.info("📊 **Recommendation Method:** Transaction-based (Items appearing with ALL your cart items)")
            st.write("💡 **Why?** No association rules matched your cart. Finding items that appeared with ALL your cart items together in transactions.")
            if cart_size > 1:
                st.warning(f"⚠️ **Note:** With {cart_size} items in your cart, fewer transactions contain all items together. Recommendations may be limited.")
        elif fallback_type == "items_appearing_with_any_cart_item":
            st.info("📊 **Recommendation Method:** Transaction-based (Items appearing with ANY of your cart items)")
            st.write("💡 **Why?** No transactions found containing all your cart items together. Showing items that appear with ANY cart item.")
            if cart_size > 1:
                st.warning(f"⚠️ **Note:** As your cart grows ({cart_size} items), exact matches become rarer. Using a more flexible matching strategy.")
        elif fallback_type == "popular_items":
            st.warning("📊 **Recommendation Method:** Popular Items (General Recommendations)")
            st.write("💡 **Why?** No transactions in the dataset contain your current cart combination.")
            st.write("Showing most popular items from the dataset instead.")
            if cart_size > 1:
                st.info(f"💡 **Insight:** Your cart has {cart_size} items. The more items you add, the fewer transactions match exactly. This is normal for market basket analysis!")
        else:
            st.info("📊 **Recommendation Method:** Transaction-based co-occurrence (fallback method used)")
            st.write("These recommendations are based on items that frequently appear together with your cart items in transactions.")
        st.write("")

        for item in recommended_items:
            st.markdown(f"### {item}")
            count = transaction_counts.get(item, 0)
            st.write(f"**Appears with your cart items in {count} transaction(s)**")

            # Calculate percentage
            total_transactions = len(st.session_state.transactions) if 'transactions' in st.session_state else 0
            if total_transactions > 0:
                percentage = (count / total_transactions) * 100
                st.write(f"**Frequency:** {percentage:.2f}% of all transactions")
            st.divider()
    else:
        st.success("✅ **Recommendation Method:** Association Rules (Apriori Algorithm)")
        st.write("**Matching Strategy:** Only rules where **ALL items in the antecedent** are present in your cart are considered.")
        st.write("")
        st.write("**Metrics Explained:**")
        st.write("- **Confidence:** Probability that consequent appears given antecedent")
        st.write("- **Lift:** How much more likely consequent is given antecedent vs. general probability")
        st.write("- **Support:** Frequency of the itemset in all transactions")
        st.write("")
        if cart_size > 1:
            st.info(f"💡 **Note:** With {cart_size} items in your cart, the system only uses rules where all {cart_size} items appear together in the antecedent. As you add more items, recommendations naturally narrow down because fewer rules match all items simultaneously.")
        st.write("")

        for item in recommended_items:
            st.markdown(f"### 🎯 {item}")

            if item in rules_used and len(rules_used[item]) > 0:
                st.write(f"**Supported by {len(rules_used[item])} association rule(s):**")

                for i, rule_info in enumerate(rules_used[item], 1):
                    st.markdown(f"#### Rule {i}:")
                    antecedents = rule_info['antecedents']
                    antecedents_str = ", ".join(antecedents)

                    # Highlight which cart items matched
                    cart_items_matched = []
                    for cart_item in cart:
                        cart_lower = cart_item.lower()
                        for ant in antecedents:
                            if cart_lower in ant.lower() or ant.lower() in cart_lower:
                                cart_items_matched.append(cart_item)
                                break

                    if cart_items_matched:
                        matched_str = ", ".join(cart_items_matched)
                        st.write(f"**Cart items matched:** {matched_str}")

                    st.write(f"**Rule:** `{antecedents_str}` → `{item}`")

                    col1, col2, col3 = st.columns(3)
                    with col1:
                        conf = rule_info.get('confidence', 0)
                        st.metric("Confidence", f"{conf:.1%}" if conf != 'N/A' else 'N/A')
                    with col2:
                        lift = rule_info.get('lift', 0)
                        st.metric("Lift", f"{lift:.2f}" if lift != 'N/A' else 'N/A')
                    with col3:
                        support = rule_info.get('support', 0)
                        st.metric("Support", f"{support:.1%}" if support != 'N/A' else 'N/A')

                    # Interpretation
                    if lift != 'N/A' and isinstance(lift, (int, float)):
                        if lift > 1:
                            st.success(f"✓ Lift > 1: This item is {lift:.2f}x more likely to appear with your cart items!")
                        elif lift == 1:
                            st.info("○ Lift = 1: This item appears independently of your cart items")
                        else:
                            st.warning(f"⚠ Lift < 1: This item is less likely to appear with your cart items")

                    if i < len(rules_used[item]):
                        st.write("---")
            else:
                st.warning("No detailed rule information available for this recommendation.")

            if item != recommended_items[-1]:
                st.divider()

def recommend_from_transactions(cart, transactions, top_n=3, require_all=False, return_counts=False):
    """
    Fallback recommendation: Find items that frequently appear with cart items in transactions.

    Parameters:
        cart: List of items in cart
        transactions: List of transaction lists
        top_n: Number of recommendations
        require_all: If True, transaction must contain ALL cart items. If False, ANY cart item.
        return_counts: If True, return tuple (items, counts_dict)

    Returns:
        List of recommended items, or tuple if return_counts=True
    """
    if len(cart) == 0 or len(transactions) == 0:
        return [] if not return_counts else ([], {})

    # Normalize cart items
    cart_set = {item.lower().strip() for item in cart}

    # Count items that appear with cart items in transactions
    item_counts = {}

    for transaction in transactions:
        transaction_set = {item.lower().strip() for item in transaction}

        # Check transaction match based on require_all flag
        if require_all:
            # Transaction must contain ALL cart items
            cart_in_transaction = cart_set.issubset(transaction_set)
        else:
            # Transaction must contain ANY cart item
            cart_in_transaction = bool(cart_set.intersection(transaction_set))

        if cart_in_transaction:
            # Count other items in this transaction
            for item in transaction:
                item_norm = item.lower().strip()

                # Skip if item is already in cart
                if item_norm not in cart_set:
                    # Check if it's really not in cart (partial match)
                    in_cart = False
                    for cart_item in cart:
                        cart_norm = cart_item.lower().strip()
                        if item_norm == cart_norm or item_norm in cart_norm or cart_norm in item_norm:
                            in_cart = True
                            break

                    if not in_cart:
                        item_counts[item] = item_counts.get(item, 0) + 1

    # Sort by frequency and return top N
    sorted_items = sorted(item_counts.items(), key=lambda x: x[1], reverse=True)
    result = [item for item, count in sorted_items[:top_n]]

    if return_counts:
        counts_dict = {item: count for item, count in sorted_items[:top_n]}
        return result, counts_dict

    return result

def get_popular_items(transactions, cart, top_n=3):
    """
    Get most popular items from the dataset (excluding cart items).
    Used as final fallback when no transaction-based recommendations are available.
    """
    if len(transactions) == 0:
        return []

    # Count all items
    item_counts = {}
    cart_set = {item.lower().strip() for item in cart}

    for transaction in transactions:
        for item in transaction:
            item_norm = item.lower().strip()
            # Skip cart items
            if item_norm not in cart_set:
                # Check partial matches
                in_cart = False
                for cart_item in cart:
                    cart_norm = cart_item.lower().strip()
                    if item_norm == cart_norm or item_norm in cart_norm or cart_norm in item_norm:
                        in_cart = True
                        break
                if not in_cart:
                    item_counts[item] = item_counts.get(item, 0) + 1

    # Sort and return top N
    sorted_items = sorted(item_counts.items(), key=lambda x: x[1], reverse=True)
    return [item for item, count in sorted_items[:top_n]]

# Initialize session state
if 'cart' not in st.session_state:
    st.session_state.cart = []
if 'recommended_items' not in st.session_state:
    st.session_state.recommended_items = []
if 'recommendation_details' not in st.session_state:
    st.session_state.recommendation_details = {}

# Load data and generate rules
# Try to find CSV file in current directory (for Colab)
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
if csv_files:
    data_file = csv_files[0]
    print(f"Using CSV file: {data_file}")
else:
    data_file = "CMPE256_Hackathon_market_basket_analysis_Release.csv"

try:
    with st.spinner("Loading data and generating association rules..."):
        transactions, all_items = load_and_preprocess_data(data_file)
        # Use very low min_support to get many rules, and lower min_confidence for better coverage
        rules_df = generate_association_rules(transactions, min_support=0.005, min_confidence=0.1)

        # Store in session state for easy access
        st.session_state.transactions = transactions
        st.session_state.all_items = all_items
        st.session_state.rules_df = rules_df

        # Compute initial recommendations if cart has items
        if len(st.session_state.cart) > 0:
            if 'recommended_items' not in st.session_state or len(st.session_state.recommended_items) == 0:
                result = recommend_items(
                    st.session_state.cart,
                    st.session_state.rules_df,
                    transactions=st.session_state.transactions,
                    top_n=3,
                    return_details=True
                )
                st.session_state.recommended_items, st.session_state.recommendation_details = result
except Exception as e:
    st.error(f"Error loading data: {str(e)}")
    st.stop()

# Header Section
st.markdown("""
<div class="main-header">
    <div style="display: flex; justify-content: space-between; align-items: center;">
        <div>
            <h3 style="margin: 0; color: white;">SJSU</h3>
            <p style="margin: 0; font-size: 12px; color: white;">SAN JOSÉ STATE UNIVERSITY</p>
        </div>
        <div style="flex-grow: 1; text-align: center;">
            <h1 style="margin: 0; color: white;">Electronic Item Checkout Web Portal</h1>
        </div>
        <div style="font-size: 24px;">🛒</div>
    </div>
</div>
""", unsafe_allow_html=True)

# Main Content Layout
col1, col2 = st.columns([1, 1.5])

with col1:
    st.markdown('<div class="section-header">Added Items to Cart:</div>', unsafe_allow_html=True)

    if len(st.session_state.cart) == 0:
        st.info("🛒 Your cart is empty. Add items to see recommendations!")
        # Show initial state message
        st.markdown("""
        <div style="background-color: #003767; color: white; padding: 15px; border-radius: 5px; margin-top: 10px;">
            <strong>New User Comes to Portal</strong>
        </div>
        """, unsafe_allow_html=True)
    else:
        for i, item in enumerate(st.session_state.cart):
            st.markdown(f'<div class="cart-item"><strong>📦 {item}</strong></div>', unsafe_allow_html=True)

with col2:
    st.markdown('<div class="section-header">Select Item from the List:</div>', unsafe_allow_html=True)

    # Dropdown for item selection
    selected_item = st.selectbox(
        "Choose an item to add:",
        options=all_items,
        index=0,
        label_visibility="collapsed"
    )

    # Add Item button
    if st.button("Add Item", type="primary"):
        if selected_item not in st.session_state.cart:
            st.session_state.cart.append(selected_item)
            # Update recommendations (will use transaction fallback if rules don't match)
            result = recommend_items(
                st.session_state.cart,
                st.session_state.rules_df,
                transactions=st.session_state.transactions,
                top_n=3,
                return_details=True
            )
            st.session_state.recommended_items, st.session_state.recommendation_details = result
            st.rerun()
        else:
            st.warning(f"{selected_item} is already in your cart!")

# Recommended Items Section
st.divider()
st.markdown('<div class="section-header">💡 Recommended Items:</div>', unsafe_allow_html=True)

if len(st.session_state.cart) == 0:
    st.info("Add items to your cart to see personalized recommendations!")
else:
    # Show what recommendations are based on
    cart_items_text = " and ".join([f"{item}" for item in st.session_state.cart])
    st.markdown(f"**Next Items list based on:** {cart_items_text}")

    if len(st.session_state.recommended_items) > 0:
        st.markdown("<br>", unsafe_allow_html=True)
        for i, item in enumerate(st.session_state.recommended_items, 1):
            st.markdown(f'<div class="recommended-item">✅ <strong>{i}.</strong> <strong>{item}</strong></div>', unsafe_allow_html=True)

        # Add validation/evidence section
        st.markdown("<br>", unsafe_allow_html=True)
        with st.expander("🔍 Validate Recommendations - See Why These Items Were Recommended", expanded=False):
            display_recommendation_validation(
                st.session_state.recommended_items,
                st.session_state.recommendation_details,
                st.session_state.cart
            )
    else:
        cart_size = len(st.session_state.cart)
        cart_items_text = " and ".join([f"**{item}**" for item in st.session_state.cart])

        st.info("📭 **No further recommendations available — no historical transaction includes all items in the current cart.**")
        st.write("")
        st.write(f"**Your cart contains:** {cart_items_text}")
        st.write("")

        if cart_size > 1:
            st.write(f"**Why no recommendations:**")
            st.write("- The system filters association rules to only consider those where **all antecedent items** are fully contained within your cart")
            st.write(f"- With {cart_size} items in your cart, no association rules have antecedents that are a complete subset of your cart items")
            st.write("- This means no historical transaction in the dataset contains all the items you currently have in your cart together")
            st.write("- This is expected behavior in market basket analysis - as cart combinations become more specific, matching rules become rarer")
            st.write("")
            st.write("**💡 Try:** Removing an item from your cart to see recommendations based on the remaining items.")
        else:
            st.write("**Why no recommendations:**")
            st.write("- No association rules in the dataset have this item as an antecedent that is fully contained in your cart")
            st.write("- No historical transaction includes this item combination")
            st.write("")
            st.write("**💡 Try:** Adding a different item or removing items to find matching patterns.")

# Optional: Clear Cart button
if len(st.session_state.cart) > 0:
    st.divider()
    if st.button("🗑️ Clear Cart", type="secondary"):
        st.session_state.cart = []
        st.session_state.recommended_items = []
        st.session_state.recommendation_details = {}
        st.rerun()

# Footer - Display some statistics
with st.expander("📊 Dataset Statistics", expanded=False):
    st.write(f"**Total Transactions:** {len(transactions)}")
    st.write(f"**Total Unique Items:** {len(all_items)}")
    st.write(f"**Frequent Itemsets Found:** {len(st.session_state.rules_df) if len(st.session_state.rules_df) > 0 else 0}")
    if len(st.session_state.rules_df) > 0:
        st.write(f"**Sample Rules:**")
        # Display rules in a more readable format
        display_rules = st.session_state.rules_df[['antecedents', 'consequents', 'confidence', 'lift']].head(10).copy()
        display_rules['antecedents'] = display_rules['antecedents'].apply(lambda x: ', '.join([str(i) for i in list(x)]))
        display_rules['consequents'] = display_rules['consequents'].apply(lambda x: ', '.join([str(i) for i in list(x)]))
        st.dataframe(display_rules)

        # Debug info for current cart
        if len(st.session_state.cart) > 0:
            st.write("**Debug Info:**")
            st.write(f"Cart items: {st.session_state.cart}")
            st.write(f"Number of recommendations found: {len(st.session_state.recommended_items)}")



Writing app.py


## Step 4: Run Streamlit App

This will start the Streamlit app. Choose one of the methods below to access it.

**Method 1 (Recommended):** Use Colab's built-in port forwarding (no authentication needed)  
**Method 2:** Use localtunnel (free, no authentication needed)  
**Method 3:** Use ngrok (requires free account and token)

In [ ]:
# METHOD 1: Using Colab's Built-in Port Forwarding (RECOMMENDED - No authentication needed)
import subprocess
import time
from IPython.display import HTML, display, Javascript
from google.colab.output import eval_js
import getpass

# Start Streamlit in background
print("🚀 Starting Streamlit app...")
print("⏳ Please wait 5-10 seconds for the app to initialize...")

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.headless", "true", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to start
time.sleep(8)

print("\n✅ Streamlit app is running!")
print("\n" + "="*60)
print("🌐 ACCESS YOUR APP:")
print("="*60)

# Method 1: Colab's built-in proxy (best for Colab)
try:
    # Get the public URL using Colab's proxy
    proxy_url = eval_js("google.colab.kernel.proxyPort(8501)")
    print(f"\n✅ Method 1 (Colab Proxy): {proxy_url}")
    display(HTML(f'<h2><a href="{proxy_url}" target="_blank">🚀 Click here to open the app!</a></h2>'))
except Exception as e:
    print(f"\n⚠️ Colab proxy not available: {e}")

    # Method 2: Use localtunnel (free, no auth)
    print("\n📡 Trying Method 2: Using localtunnel...")
    try:
        import subprocess as sp
        sp.run(["pip", "install", "-q", "localtunnel"], check=False)
        import threading

        def run_tunnel():
            sp.run(["npx", "localtunnel", "--port", "8501"], check=False)

        tunnel_thread = threading.Thread(target=run_tunnel, daemon=True)
        tunnel_thread.start()
        time.sleep(5)

        print("\n✅ Localtunnel started!")
        print("📋 Look for the 'your url is:' message above and click that link")
        print("💡 Or check the output for a URL starting with 'https://'")
    except Exception as e2:
        print(f"\n⚠️ Localtunnel failed: {e2}")

        # Method 3: Manual instructions
        print("\n📝 Method 3: Manual Access")
        print("="*60)
        print("\nThe Streamlit app is running on port 8501.")
        print("\nTo access it:")
        print("1. Look for 'Connect' or 'Preview' button in the Colab output")
        print("2. Or use Colab's port forwarding feature")
        print("3. Or try accessing via: http://localhost:8501")
        print("\n💡 Tip: Right-click on the notebook and select 'Change runtime type'")
        print("   then enable 'Port forwarding' if available.")

print("\n" + "="*60)
print("⚠️ Note: The app will stop when you interrupt the kernel or restart the runtime.")
print("="*60)

🚀 Starting Streamlit app...
⏳ Please wait 5-10 seconds for the app to initialize...

✅ Streamlit app is running!

🌐 ACCESS YOUR APP:

✅ Method 1 (Colab Proxy): https://8501-gpu-t4-s-3jztzjh9zfkl7-c.asia-southeast1-2.prod.colab.dev



⚠️ Note: The app will stop when you interrupt the kernel or restart the runtime.


## Step 5: Alternative - Direct Access Method

If the methods above don't work, use this simpler approach:


In [ ]:
# SIMPLE METHOD: Run Streamlit and Access via Colab Preview
import subprocess
import time
from IPython.display import IFrame, display, HTML

print("🚀 Starting Streamlit app...")
print("⏳ Please wait 8-10 seconds...")

# Start Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.headless", "true", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(8)

print("\n" + "="*70)
print("✅ Streamlit app is running!")
print("="*70)

# Try to embed using Colab's output utilities
try:
    from google.colab.output import eval_js, register_callback

    # Get the proxy URL
    def get_url():
        return eval_js("google.colab.kernel.proxyPort(8501, {'cache': false})")

    url = get_url()
    print(f"\n🌐 Your app URL: {url}")
    print("\n📱 Access methods:")
    print(f"   1. Click this link: {url}")
    print(f"   2. Or open it in a new tab")

    # Display as clickable link
    display(HTML(f'''
    <div style="padding: 20px; background-color: #e8f5e9; border-radius: 10px; margin: 20px 0;">
        <h2>🚀 Click to Open Your App!</h2>
        <p><a href="{url}" target="_blank" style="font-size: 18px; color: #1976d2; text-decoration: none;">
            {url}
        </a></p>
        <p><em>Note: If the link doesn't work, copy the URL above and paste it in a new browser tab.</em></p>
    </div>
    '''))

except Exception as e:
    print(f"\n⚠️ Auto-link failed: {e}")
    print("\n📋 MANUAL ACCESS INSTRUCTIONS:")
    print("="*70)
    print("\n1. Look for a 'Connect' or 'Preview' button in the output above")
    print("2. Or right-click on this notebook cell output")
    print("3. Select 'Open in new tab' or similar option")
    print("4. Or manually navigate to: http://localhost:8501")
    print("\n💡 The app is running on port 8501")
    print("💡 You may need to enable port forwarding in Colab settings")

print("\n" + "="*70)
print("✅ App is ready! Add items to your cart and see recommendations.")
print("="*70)


🚀 Starting Streamlit app...
⏳ Please wait 8-10 seconds...

✅ Streamlit app is running!

🌐 Your app URL: https://8501-gpu-t4-s-3jztzjh9zfkl7-c.asia-southeast1-2.prod.colab.dev

📱 Access methods:
   1. Click this link: https://8501-gpu-t4-s-3jztzjh9zfkl7-c.asia-southeast1-2.prod.colab.dev
   2. Or open it in a new tab



✅ App is ready! Add items to your cart and see recommendations.


## Usage Instructions

1. **Upload CSV**: Run the upload cell (Step 2) to add your dataset
2. **Create App**: Run Step 3 to generate the app.py file
3. **Run Streamlit**: Run Step 4 to start the app and get a public URL
4. **Access App**: Click the public URL provided to open the app in your browser
5. **Test Recommendations**: Add items to cart and see real-time recommendations

### Test Case:
Try adding these 3 items to test the system:
1. GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)
2. Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-IPC-HDBW4431R-ZS)
3. Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)

**Expected Recommendation:** Hikvision 4MP IP Camera (SKU: DS-2CD2142FWD-I)

---

**Note:** To stop the app, interrupt the kernel or restart the runtime.